<a href="https://colab.research.google.com/github/luissanty2/TFGRecursos/blob/main/Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_scores1 = pd.read_csv("match_scores_1991-2016_unindexed_csv.csv", encoding = 'latin1')
df_scores2 = pd.read_csv("match_scores_2017_unindexed_csv.csv", encoding = 'latin1')

df_scores = pd.concat([df_scores1, df_scores2], axis=0)


df_stats1 = pd.read_csv("match_stats_1991-2016_unindexed_csv.csv", encoding = 'latin1')
df_stats2 = pd.read_csv("match_stats_2017_unindexed_csv.csv", encoding = 'latin1')

df_stats = pd.concat([df_stats1, df_stats2], axis=0)


df_stats['winner_double_faults'] = df_stats['winner_double_faults'] / df_stats['winner_service_points_total']
df_stats['loser_double_faults'] = df_stats['loser_double_faults'] / df_stats['loser_service_points_total']

df_stats['winner_first_serves_in'] = df_stats['winner_first_serves_in'] / df_stats['winner_first_serves_total']
df_stats['loser_first_serves_in'] = df_stats['loser_first_serves_in'] / df_stats['loser_first_serves_total']

df_stats['winner_first_serve_points_won'] = df_stats['winner_first_serve_points_won'] / df_stats['winner_first_serve_points_total']
df_stats['loser_first_serve_points_won'] = df_stats['loser_first_serve_points_won'] / df_stats['loser_first_serve_points_total']

df_stats['winner_second_serves_in'] = 1 - (df_stats['winner_first_serves_in'] + df_stats['winner_double_faults'])
df_stats['loser_second_serves_in'] = 1 - (df_stats['loser_first_serves_in'] + df_stats['loser_double_faults'])


df_matchs = pd.merge(df_stats[['match_id','match_duration','winner_aces', 'loser_aces','winner_double_faults','loser_double_faults','winner_first_serves_in','loser_first_serves_in', 'loser_first_serve_points_won', 'winner_first_serve_points_won',
                               'winner_second_serves_in', 'loser_second_serves_in']],
                     df_scores[['tourney_year_id','match_id','tourney_round_name','winner_name','winner_player_id','loser_name','loser_player_id','winner_sets_won','loser_sets_won','winner_games_won','loser_games_won']],
                     on='match_id')


df_tournaments = pd.read_csv("tournaments_1877-2017_unindexed_csv.csv", encoding = 'latin1')


df = pd.merge(df_matchs, df_tournaments[['tourney_surface','tourney_year_id','tourney_dates','tourney_year', 'tourney_month']], on=['tourney_year_id'])

print(df.shape)



(95773, 26)


In [ ]:
df_ranking = pd.read_csv('rankings_1973-2017_csv.csv')


df = pd.merge(df, df_ranking[['ranking_points', 'week_year', 'week_month','player_id']], how='left', left_on=['tourney_year', 'tourney_month', 'winner_player_id'], right_on=['week_year','week_month','player_id'])
df['winner_ranking_points'] = df['ranking_points']


df = df.drop(columns=['ranking_points', 'week_year', 'week_month','player_id'])


df = pd.merge(df, df_ranking[['ranking_points', 'week_year', 'week_month','player_id']], how='left', left_on=['tourney_year', 'tourney_month', 'loser_player_id'], right_on=['week_year','week_month','player_id'])
df['loser_ranking_points'] = df['ranking_points']

df = df.drop(columns=['ranking_points', 'week_year', 'week_month','player_id'])

df = df.drop_duplicates(subset='match_id', keep="first")



df = df.drop(df[df.tourney_year < 2000].index)
df = df.drop(df[df.tourney_surface == "Carpet"].index)
df = df.drop(columns=['tourney_year','match_id','tourney_year_id', 'winner_player_id', 'loser_player_id','tourney_month', 'tourney_dates'])
df = df.dropna()
df.rename(columns={"tourney_surface": "surface"}, inplace=True)

print(df.shape)

<ipython-input-123-0f9b17f9398c>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ranking = pd.read_csv('rankings_1973-2017_csv.csv')


(63181, 21)


<ipython-input-123-0f9b17f9398c>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"tourney_surface": "surface"}, inplace=True)


In [ ]:
print(df.shape)
print(df.columns)
df.to_csv('dataset_final.csv')

(63181, 21)
Index(['match_duration', 'winner_aces', 'loser_aces', 'winner_double_faults',
       'loser_double_faults', 'winner_first_serves_in',
       'loser_first_serves_in', 'loser_first_serve_points_won',
       'winner_first_serve_points_won', 'winner_second_serves_in',
       'loser_second_serves_in', 'tourney_round_name', 'winner_name',
       'loser_name', 'winner_sets_won', 'loser_sets_won', 'winner_games_won',
       'loser_games_won', 'surface', 'winner_ranking_points',
       'loser_ranking_points'],
      dtype='object')
